# **A simple web application with nglview and periodic table widget**

* textarea to input name of the molecules (ipywidgets)
* obtain molecular structure from ase.build.molecule (ase)
* show the structure with nglview (nglview)
* highlight the elements of the strucutre in the periodic table (widget_periodictable)
* use Voila and OSSCAR template to present the web app (voila and voila-osscar-template)

In [1]:
from ase.build import molecule
from widget_periodictable import PTableWidget
import nglview as nv
from ipywidgets import HBox, VBox, Textarea, Button, Layout
import numpy as np
from vapory import *
from numpy.linalg import norm
from copy import deepcopy
from ase.io import read, write

In [2]:
colors = {'H': [255,255,255], 
          'He': [217,255,255],
          'Li': [204,128,255],
          'Be': [194,255,0],
          'B': [255,181,181],
          'C': [144,144,144],
          'N': [48,80,248],
          'O': [255,13,13],
          'F': [144,224,80],
          'Ne': [179,227,245],
          'Na': [171,92,242],
          'Mg': [138,255,0],
          'Al': [191,166,166],
          'Si': [240,200,160],
          'P': [255,128,0],
          'S': [255,255,48],
          'Cl': [31,240,31],
          'Ar': [128,209,227],
          'K': [143,64,212],
          'Ca': [61,255,0],
          'Sc': [230,230,230],
          'Ti': [191,194,199],
          'V': [166,166,171],
          'Cr': [138,153,199],
          'Mn': [156,122,199],
          'Fe': [224,102,51],
          'Co': [240,144,160],
          'Ni': [80,208,80],
          'Cu': [200,128,51],
          'Zn': [125,128,176],
          'Ga': [194,143,143],
          'Ge': [102,143,143],
          'As': [189,128,227],
          'Se': [255,161,0],
          'Br': [166,41,41],
          'Kr': [92,184,209],
          'Rb': [112,46,176],
          'Sr': [0,255,0],
          'Y': [148,255,255],
          'Zr': [148,224,224],
          'Nb': [115,194,201],
          'Mo': [84,181,181],
          'Tc': [59,158,158],
          'Ru': [36,143,143],
          'Rh': [10,125,140],
          'Pd': [0,105,133],
          'Ag': [192,192,192],
          'Cd': [255,217,143],
          'In': [166,117,115],
          'Sn': [102,128,128],
          'Sb': [158,99,181],
          'Te': [212,122,0],
          'I': [148,0,148],
          'Xe': [66,158,176],
          'Cs': [87,23,143],
          'Ba': [0,201,0],
          'La': [112,212,255],
          'Ce': [255,255,199],
          'Pr': [217,255,199],
          'Nd': [199,255,199],
          'Pm': [163,255,199],
          'Sm': [143,255,199],
          'Eu': [97,255,199],
          'Gd': [69,255,199],
          'Tb': [48,255,199],
          'Dy': [31,255,199],
          'Ho': [0,255,156],
          'Er': [0,230,117],
          'Tm': [0,212,82],
          'Yb': [0,191,56],
          'Lu': [0,171,36],
          'Hf': [77,194,255],
          'Ta': [77,166,255],
          'W': [33,148,214],
          'Re': [38,125,171],
          'Os': [38,102,150],
          'Ir': [23,84,135],
          'Pt': [208,208,224],
          'Au': [255,209,35],
          'Hg': [184,184,208],
          'Tl': [166,84,77],
          'Pb': [87,89,97],
          'Bi': [158,79,181],
          'Po': [171,92,0],
          'At': [117,79,69],
          'Rn': [66,130,150],
          'Fr': [66,0,102],
          'Ra': [0,125,0],
          'Ac': [112,171,250],
          'Th': [0,186,255],
          'Pa': [0,161,255],
          'U': [0,143,255],
          'Np': [0,128,255],
          'Pu': [0,107,255],
          'Am': [84,92,242],
          'Cm': [120,92,227],
          'Bk': [138,79,227],
          'Cf': [161,54,212],
          'Es': [179,31,212],
          'Fm': [179,31,186],
          'Md': [179,13,166],
          'No': [189,13,135],
          'Lr': [199,0,102],
          'Rf': [204,0,89],
          'Db': [209,0,79],
          'Sg': [217,0,69],
          'Bh': [224,0,56],
          'Hs': [230,0,46],
          'Mt': [235,0,38]
         }

radius = {'H': 0.53,
          'He': 0.31,
          'Li': 1.67,
          'Be': 1.12,
          'B': 0.87,
          'C': 0.67,
          'N': 0.56,
          'O': 0.48,
          'F': 0.42,
          'Ne': 0.38,
          'Na': 1.90,
          'Mg': 1.45,
          'Al': 1.18,
          'Si': 1.11,
          'P': 0.98,
          'S': 0.88,
          'Cl': 0.79,
          'Ar': 0.71,
          'K': 2.43,
          'Ca': 1.94,
          'Sc': 1.84,
          'Ti': 1.76,
          'V': 1.71,
          'Cr': 1.66,
          'Mn': 1.61,
          'Fe': 1.56,
          'Co': 1.52,
          'Ni': 1.49,
          'Cu': 1.45,
          'Zn': 1.42,
          'Ga': 1.36,
          'Ge': 1.25,
          'As': 1.14,
          'Se': 1.03,
          'Br': 0.94,
          'Kr': 0.88,
          'Rb': 2.65,
          'Sr': 2.19,
          'Y': 2.12,
          'Zr': 2.06,
          'Nb': 1.98,
          'Mo': 1.90,
          'Tc': 1.83,
          'Ru': 1.78,
          'Rh': 1.73,
          'Pd': 1.69,
          'Ag': 1.65,
          'Cd': 1.61,
          'In': 1.56,
          'Sn': 1.45,
          'Sb': 1.33,
          'Te': 1.23,
          'I': 1.15,
          'Xe': 1.08,
          'Cs': 2.98,
          'Ba': 2.53,
          'La': 1.95,
          'Ce': 1.85,
          'Pr': 2.47,
          'Nd': 2.06,
          'Pm': 2.05,
          'Sm': 2.38,
          'Eu': 2.31,
          'Gd': 2.33,
          'Tb': 2.25,
          'Dy': 2.28,
          'Ho': 2.26,
          'Er': 2.26,
          'Tm': 2.22,
          'Yb': 2.22,
          'Lu': 2.17,
          'Hf': 2.08,
          'Ta': 2.00,
          'W': 1.93,
          'Re': 1.88,
          'Os': 1.85,
          'Ir': 1.80,
          'Pt': 1.77,
          'Au': 1.74,
          'Hg': 1.71,
          'Tl': 1.56,
          'Pb': 1.54,
          'Bi': 1.43,
          'Po': 1.35,
          'At': 1.27,
          'Rn': 1.20,
          'Fr': 1.94,
          'Ra': 1.62,
          'Ac': 1.95,
          'Th': 1.80,
          'Pa': 1.80,
          'U': 1.75,
          'Np': 1.75,
          'Pu': 1.75,
          'Am': 1.75,
          'Cm': 1.75,
          'Bk': 1.75,
          'Cf': 1.75,
          'Es': 1.75,
          'Fm': 1.75,
          'Md': 1.75,
          'No': 1.75,
          'Lr': 1.75,
          'Rf': 1.75,
          'Db': 1.75,
          'Sg': 1.75,
          'Bh': 1.75,
          'Hs': 1.75,
          'Mt': 1.75
    }

In [3]:
aa = molecule("C60")
aa.set_cell([[15, 0, 0], [0, 15, 0], [0, 0, 15]])
aa.center()
aa.pbc=True
view = nv.show_ase(aa)
view.add_unitcell()
view.control.zoom(0.2)
view.add_ball_and_stick(aspectRatio=4)
view.camera='perspective'

In [4]:
w = Textarea(
    value='C60',
    placeholder='Type your molecule',
    description='Molecule:',
    disabled=False,
    layout=Layout(width='28%', height='27px')
)

In [5]:
PTable = PTableWidget(states=1, selected_colors = ['red'], selected_elements = {'C':0}, 
                      border_color='black', unselected_color = 'pink', width='20px')

In [6]:
def on_button_click(b):
    global view, aa
    aa = molecule(w.value)
    aa.set_cell([[15, 0, 0], [0, 15, 0], [0, 0, 15]])
    aa.center()
    aa.pbc=True
    for comp_id in view._ngl_component_ids:
        view.remove_component(comp_id)
    view.add_component(nv.ASEStructure(aa))
    view.clear()
    view.add_ball_and_stick(aspectRatio=4)
    view.add_unitcell()
    view.control.zoom(0.2)
    PTable.selected_elements = {key: 0 for key in list(dict.fromkeys(aa.get_chemical_symbols()))}

def povray_render(b):
    A = view._camera_orientation
    A = np.array(A)
    A=A.reshape(4,4)
    A=np.transpose(A)

    zfactor = norm(A[0, 0:3])
    A[0:3, 0:3] = A[0:3, 0:3]/zfactor
    
    bb = deepcopy(aa);

    for i in bb:
        a = np.array([i.x, i.y, i.z])
        a = a + A[0:3, 3];
        w = A[0:3, 0:3].dot(a)
        i.x = -w[0]
        i.y = w[1]
        i.z = w[2]
        
    vertices = [];
    
    vx = bb.get_cell()[0][0];
    vy = bb.get_cell()[1][1];
    vz = bb.get_cell()[2][2];

    vertices.append(np.array([0, 0, 0]));
    vertices.append(np.array([0, vy, 0]));
    vertices.append(np.array([0, 0, vz]));
    vertices.append(np.array([0, vy, vz]));
    
    vertices.append(np.array([vx, 0, 0]));
    vertices.append(np.array([vx, vy, 0]));
    vertices.append(np.array([vx, 0, vz]));
    vertices.append(np.array([vx, vy, vz]));

    for n, i in enumerate(vertices):
        a = i + A[0:3, 3];
        w = A[0:3, 0:3].dot(a)
        vertices[n] = np.array([-w[0], w[1], w[2]])
    

    camera = Camera('location', [0, 0, -zfactor/1.5], 'look_at', [0.0, 0.0, 0.0])

    light1 = LightSource([0, 0, -100.0], 'color',  [1.5, 1.5, 1.5])
    light2 = LightSource([-100.0, -100.0, -60], 'color',  [1.5, 1.5, 1.5])
    light3 = LightSource([0, -60.0, 0], 'color',  [1, 1, 1])

    #light = LightSource(-A[2][0:3], [1.3, 1.3, 1.3])
    wall = Plane([0, 0, 100], 20, Texture(Pigment('color', [1, 1, 1])))

    spheres = [];

    for i in bb:
        sphere = Sphere( [i.x, i.y, i.z], radius[i.symbol], 
                        Texture(Pigment( 'color', np.array(colors[i.symbol])/255)), 
                            Finish('phong', 0.9,'reflection', 0.05))
        spheres.append(sphere)


    bonds = [];
    for x, i in enumerate(bb):
        for j in bb[x+1:]:
            v1 = np.array([i.x, i.y, i.z])
            v2 = np.array([j.x, j.y, j.z])

            if norm(v1-v2) < 1.7:
                bond = Cylinder(v1, v2, 0.2, Pigment('color', (np.array(colors[i.symbol])+np.array(colors[j.symbol]))/510),
                                Finish('phong', 0.8,'reflection', 0.5))
                bonds.append(bond)
                
    edges = [];
    for x, i in enumerate(vertices):
        for j in vertices:
            if norm(i-j) < (vx+0.01) and norm(i-j) > (vx-0.01):
                edge = Cylinder(i, j, 0.06, Texture('Ruby_Glass'))
                edges.append(edge)

    scene = Scene( camera, objects= [light1] + spheres + bonds + [Background( "color", [1,1,1])])
    #scene = Scene( camera, objects= [light1, wall] + spheres + bonds, included=["textures.inc"] )
    scene.render("nglview.png", width=4000, height=4000, antialiasing=0.000, quality=11, remove_temp=False)

    
b = Button(description = 'Update')
br = Button(description = 'Render')
b.on_click(on_button_click)
br.on_click(povray_render)

In [7]:
display(HBox([w, b, br]), VBox([view, PTable]))